# Tutorial on Interrupted Time Series Analysis with Prophet on Nigeria data

This tutorial pertains to training and evaluating an interrupted time series model in python using the open-ource Prophet libarary by Facebook Inc. (https://facebook.github.io/prophet/)

We will walk through 6 main steps:

1) Installing and/loading loading required libraries<br>
2) Loading Nigeria dataset<br>
3) Data Preparation
3) Getting forecasts<br>
4) Tuning hyperparameters<br> 
5) Get forecasts using tuned hyperparameters<br>
6) Evaluation model performance using cross validation

## 1. Install and/or Import Required Libraries


We shall be using a Python 3.8 environment since Prophet is compatible with < 3.9

In [ ]:
# !pip install pandas
# !pip install matplotlib
# !pip install seaborn
# !pip install hampel
# !pip install scipy
# !pip install openpyxl # extension to read excel files
# !pip install pystan==2.19.1.1
# !pip install prophet --no-cache

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib as mpl 
import seaborn as sns
import pickle
import itertools
import os

from IPython.display import display, HTML
from datetime import datetime
from scipy.stats import wilcoxon, t, sem
from hampel import hampel
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('ggplot')

## 2. Loading and cleaning the Nigeria dataset

This step involves loading the Excel sheet with the Nigeria essential health services data; Antenatal Visits, maternal deaths and facility deliveries.<br>

In this work, we use Antenatal Visits and facility deliveries from January 2017 through December 2021.

#### a. Load Antenatal Visits data

anc_data_columns = ['organisationunitname', 'ANC Syphilis case treated','ANC Syphilis test done ', 'ANC Syphilis test positive ',
                      'Antenatal 1st (booking) visit 20 weeks or later','periodname',
                      'Antenatal 1st (booking) visit before 20 weeks', 'Antenatal 4th Visit','Date']

In [ ]:
ng_dhs_anc_data = pd.read_csv('anc_visits_nigeria.csv')
ng_dhs_anc_data['Date'] = pd.to_datetime(ng_dhs_anc_data['periodname'])
ng_dhs_anc_data.sort_values(by = 'Date')
ng_dhs_anc_data['Year'] = ng_dhs_anc_data.Date.dt.year
ng_dhs_anc_data['Year'].value_counts()

In [ ]:
ng_dhs_anc_data.info()

In [ ]:
ng_dhs_anc_data

#### b. Load facility deliveries data

In [ ]:
facility_deliveries_columns = ['Date','organisationunitname','Deliveries with complications - mother only',
                               'Preterm birth (<37th Weeks)','Deliveries']

In [ ]:
ng_dhs_facility_deliveries = pd.read_csv('facility_deliveries_nigeria.csv')
ng_dhs_facility_deliveries['Date'] = pd.to_datetime(ng_dhs_facility_deliveries['periodname'])
ng_dhs_facility_deliveries = ng_dhs_facility_deliveries[facility_deliveries_columns]
ng_dhs_facility_deliveries['Year'] = ng_dhs_facility_deliveries.Date.dt.year
ng_dhs_facility_deliveries['Year'].value_counts()

In [ ]:
ng_dhs_facility_deliveries.info()

- We can see that the deliveries column has a lot of missing data, thus cannot be usefull in this work

#### c. Merge all dataframes

We will merge the two dataframes; Antenatal Visits and Facility Deliveries (Deliveries with complications - mother only, Preterm births) to work with one dataframe onwards

In [ ]:
final_columns = ['Antenatal 4th Visit','Preterm birth (<37th Weeks)','Deliveries with complications - mother only']

In [ ]:
facility_anc_merge = ng_dhs_facility_deliveries.merge(ng_dhs_anc_data, how='left', on=['Date','Year','organisationunitname'])
facility_anc_merge = facility_anc_merge [['Date','Year','organisationunitname'] + final_columns ]
facility_anc_merge

#### e. Aggreagate the State level data into Regions

In [ ]:
def aggregate_to_regions (row):
    if row['organisationunitname'] in South_South:
        return 'South South'
    if row['organisationunitname'] in South_West:
        return 'South West'
    if row['organisationunitname'] in South_East:
        return 'South East'
    if row['organisationunitname'] in North_Central:
        return 'North Central'
    if row['organisationunitname'] in North_East:
        return 'North East'
    if row['organisationunitname'] in North_West:
        return 'North West'
    else:
        return 'Unknown'

In [ ]:
South_South = ["ak Akwa-Ibom State","by Bayelsa State","cr Cross River State","de Delta State","ed Edo State","ri Rivers State"] 
South_East = ["ab Abia State"," an Anambra state","eb Ebonyi State","en Enugu State","im Imo State"]
South_West = ["ek Ekiti State","la Lagos State","og Ogun State","on Ondo State","os Osun State","oy Oyo State"]
North_East = ["ad Adamawa State","ba Bauchi State","bo Borno State","go Gombe State","ta Taraba State","yo Yobe State"]
North_West = ["ji Jigawa State","kd Kaduna State","kn Kano State","kt Katsina State","ke Kebbi State","so Sokoto State","za Zamfara State"]
North_Central = ["be Benue State","ko Kogi State","kw Kwara State","na Nasarawa State","ni Niger State","pl Plateau State","fc Federal Capital Territory"]

In [ ]:
df_copy = facility_anc_merge.copy() # Make a copy of the dataframe

# Map Regions to States
df_copy['Region'] = df_copy.apply(aggregate_to_regions, axis = 1)
df_copy.head()

#### d. Remove Outliers

We need to check if we have any outliers and deal with them before running the analysis. We shall do this by visualising the data. <br>
There are other techniques to identify outliers like boxplots 

In [ ]:
from scipy import stats
def plot_sphaghetti(dat, id_var, x_var, y_var, title):
    """
    Plot sphaghetti 
    
    @param dat: data in long format (dataframe)
    @param id_var: column name of unique ids of individuals (str)
    @param x_var: name of x variable (str)
    @param y_var: name of y variable (str)
    @param sample_size: sample size (int)
    """
    
    sns.set(rc={'figure.figsize':(12,7)})
    mpl.rcParams['font.size'] = 8.0
    mpl.rcParams["font.weight"] = "bold"
    mpl.rcParams["axes.labelweight"] = "bold"

    ids = dat[id_var]
 
  
    for i in ids:
        df = dat[[x_var, y_var]][dat[id_var] == i]
        df[y_var] = df[y_var].fillna(df[y_var].mean())
        #z = np.abs(stats.zscore(df[y_var]))
        #df = df[z<3]
        plt.plot(df[x_var],df[y_var], marker='', color='black', linewidth=0.1, alpha=0.1)
    
    plt.ylabel('Count', fontsize = 18, fontweight='bold')
    #plt.xlabel(x_var, fontsize = 14, fontweight='bold')
    
    plt.title(title,fontsize = 20, fontweight='bold')
    plt.xticks(rotation=90, fontsize =18, fontweight='bold')
    plt.yticks(fontsize =18, fontweight='bold')
    if not os.path.exists('plots'):
        os.makedirs('plots')
    plt.savefig("plots/{}.png".format(title),  dpi=300, bbox_inches='tight')
    plt.xlabel('Date')
    plt.show()
    plt.close()

In [ ]:
for column in final_columns:
    title = '{} per Organizational Unit'.format(column)
    plot_sphaghetti(df_copy,'organisationunitname', 'Date', column , title)

In [ ]:
type_mappings = {'Antenatal 4th Visit':'4th Antenatal Visit',
                'Preterm birth (<37th Weeks)':'Preterm deliveries ',
                'Deliveries with complications - mother only':'Complicated Deliveries(Mother)'}

In [ ]:
# Function to remove outliers
def remove_outliers(x):
    if len(set(x)) == 1 or x.isnull().all():
        return x
    x = x.fillna(x.mean())
    x = pd.Series([int(i) for i in x])
    return hampel(x, window_size=5, n=3, imputation=True) 

In [ ]:
for typ in ['Antenatal 4th Visit']:
    df_copy[typ] = df_copy.groupby('organisationunitname')[typ].transform(remove_outliers)
    df_copy[typ] = df_copy[typ].astype(int)
df_copy.head()

In [ ]:
for column in ['Antenatal 4th Visit']:
    title = '{} per Organizational Unit'.format(column)
    plot_sphaghetti(df_copy,'organisationunitname', 'Date', column , title)

From the plot above it is visible, that the outlier has been delt with.

# 3. Data Preparation

#### a. Prepare the data for the ITS pipeline

In [ ]:
# Impute the Missing values
def fill_values(dataset,col):
    dataset[col] = dataset[col].fillna(dataset[col].rolling(8,min_periods=1).mean())
    return dataset[col]

for col in final_columns:
    fill_values(df_copy, col)

In [ ]:
df_copy.info()

In [ ]:
df_copy

In [ ]:
# Convert datatype to int
for typ in final_columns:
    df_copy[typ] = df_copy[typ].astype(int)

In [ ]:
# Calculate monthly total for each region
df_copy = df_copy.groupby(['Region','Date'])[final_columns].sum().reset_index()
df_copy = df_copy.copy()
df_copy

#### b) Set labels for easy plotting

In [ ]:
outcome_label_original = {'Antenatal 4th Visit':'Antenatal 4th Visit',
                 'Preterm birth (<37th Weeks)':'Preterm births',
                 'Deliveries with complications - mother only': 'Deliveries with complications:Mother'}

In [ ]:
#Descriptive labels for y axis
y_axis_label  = {'Antenatal 4th Visit':'4th antenatal visits (n)', 
                'Preterm birth (<37th Weeks)':'number of preterm deliveries',
                'Deliveries with complications - mother only':'number of complicated deliveries'}

In [ ]:
regions = df_copy['Region'].unique().tolist()
regions

# 4. Getting forecasts

We can now train an ITS models the ANC Visits, Preterm births and Complicated Deliveries(Mother only) data. We train the model using data up to Feb 01, 2020, and then use the model to predict the outcome for the remaining months in our data (March 2020 - December 2020). We then compare the trends in the actual and predicted outcomes

#### a) Specify prediction period

In [ ]:
prediction_start_date='2020-03-01'
prediction_end_date='2020-12-01'

#### b) Get forecasts

In [ ]:
def get_forecasts(location, outcome, data, seasonality_mode='additive', changepoint_prior_scale=0.05, seasonality_prior_scale=10.0, prediction_start_date='2020-03-01', prediction_end_date='2020-06-01'):
    
    '''
    Function to get the prophet forecasts. 
    
    @params:
    location: Geographical unit (string)
    outcome: Outcome measure (string)
    data: Dataframe with date and the outcome value for the geographical unit and outcome. (pd.DataFrame)
    seasonality_mode: tuned prophet model parameter (string)
    changepoint_prior_scale: tuned prophet model parameter (double)
    seasonality_prior_scale: tuned prophet model parameter (double)
    prediction_start_date: Prediction start date (datetime)
    prediction_end_date: Prediction end date (datetime)
    
    '''
    df = data.copy()  

    prediction_start_date = datetime.strptime(prediction_start_date, '%Y-%m-%d')
    
    prediction_end_date = datetime.strptime(prediction_end_date, '%Y-%m-%d')
    
    df = df[df['ds']<=prediction_end_date] 

    df_training = df[df['ds']<prediction_start_date] 

    df_test = df[df['ds']>=prediction_start_date]
    
    m = Prophet(interval_width=.95, 
            growth ='linear',
            yearly_seasonality=False, 
            weekly_seasonality = False, 
            daily_seasonality = False,
            seasonality_mode = seasonality_mode,
            changepoint_prior_scale = changepoint_prior_scale,
            seasonality_prior_scale = seasonality_prior_scale,
            ).add_seasonality(
                name='yearly',
                period = 365,
                fourier_order = 5
            )

    m.fit(df_training)

    future = pd.DataFrame(df['ds'], columns = ['ds'])
    
    forecast = m.predict(future)

    forecast['y'] = [i for i in df['y']]
    
    forecast['change'] = forecast['y'] - forecast['yhat']
    
    forecast['percent_change'] = forecast.apply(lambda row: round((row.y - row.yhat)/row.yhat *100,2), axis =1)
            
    
    return forecast[['ds','y','yhat','change','percent_change','yhat_lower','yhat_upper','trend','trend_lower','trend_upper']].copy()


In [ ]:
# Get forecast for each region and outcome
forecast_dict = {}
for region in regions:
    measures_forecast = {}
    for measure in final_columns:
        data = df_copy.loc[df_copy['Region'] == region].copy()[[measure,'Date','Region']]
        data = data[['Date', measure]]
        data.columns = ['ds','y']
        
        forecast = get_forecasts(location = region,
                                  outcome = measure,
                                  data = data, 
                                  prediction_start_date = prediction_start_date,
                                  prediction_end_date = prediction_end_date)
        
        measures_forecast[measure] = forecast
        
    forecast_dict[region] = measures_forecast

In [ ]:
forecast_dict

In [ ]:
forecast_dict['North Central']['Antenatal 4th Visit'].tail()

#### c. Plot Forecasts

In [ ]:
def plot_its(location, outcome, forecast, prediction_start_date, prediction_end_date, normalise = False):
    '''
    Function to plot the forecast counts.
    
    @params:
    location: Geographical unit (string)
    outcome: Outcome measure (string)
    forecast: Dataframe with the prophet forecast output for the geographical unit and outcome(pd.DataFrame)
    prediction_start_date: Prediction start date (datetime)
    prediction_end_date: Prediction end date (datetime)
    normalise: Data normalised (boolean)
    
    '''
    sns.set(rc={'figure.figsize':(10,8)})
    sns.set_style("white")
    mpl.rcParams['font.size'] = 8.0
    mpl.rcParams["font.weight"] = "bold"
    mpl.rcParams["axes.labelweight"] = "bold"

    prediction_start_date = datetime.strptime(prediction_start_date, '%Y-%m-%d')
    prediction_end_date = datetime.strptime(prediction_end_date, '%Y-%m-%d')
    
    plt.axvspan(xmin = prediction_start_date , xmax=prediction_end_date, color='grey', alpha=0.2, lw=0)
    
    plt.scatter(forecast['ds'],forecast['y'], facecolors='none', edgecolors='black', s =20, label = 'observed values')
    plt.xlim([forecast['ds'].min(),prediction_end_date])
    
    plt.plot(forecast['ds'],forecast['yhat'], color = '#33adff', label ='predicted values')
    plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], color='#33adff', alpha=0.25)

    plt.plot(forecast['ds'],forecast['trend'], color = 'red', linestyle ="--", label ='predicted trend')
    plt.fill_between(forecast['ds'], forecast['trend_lower'], forecast['trend_upper'], color='red', alpha=0.2)
    
    plt.xticks(rotation=90, fontsize =18, fontweight='bold')
    plt.yticks(fontsize =18, fontweight='bold') 
    
    plt.xlabel('') 
   
    
    if normalise:
        plt.ylabel('count per 100k', fontsize =18, fontweight='bold')
        title = '{} in {} normalised'.format(outcome_label[outcome],location)
    else:
        plt.ylabel('counts', fontsize =18, fontweight='bold')
        title = '{} in {} '.format(outcome_label_original[outcome],location)
        
    plt.title(title,fontsize = 20, fontweight='bold')
    plt.legend(loc='upper left',fontsize=12)
    plt.savefig("plots/"+title+"_its.png", dpi=300,bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
# Plot forecast for each region and outcome
for region in forecast_dict.keys():
    for measure in forecast_dict[region].keys():
        plot_its(location = region,        
                     outcome = measure,
                     forecast = forecast_dict[region][measure],
                     prediction_start_date = prediction_start_date,
                     prediction_end_date = prediction_end_date, 
                     normalise = False)

In [ ]:
def plot_cumulative(location, outcome, forecast, prediction_start_date, prediction_end_date, normalise = False):
    '''
    Function to plot the cumulative forecast.
    
    @params:
    location: Geographical unit (string)
    outcome: Outcome measure (string)
    forecast: Dataframe with the prophet forecast output for the geographical unit and outcome (pd.DataFrame)
    prediction_start_date: Prediction start date (datetime)
    prediction_end_date: Prediction end date (datetime)
    normalise: Data normalised (boolean)
    
    '''
    sns.set(rc={'figure.figsize':(10,8)})
    sns.set_style("white")
    mpl.rcParams['font.size'] = 8.0
    mpl.rcParams["font.weight"] = "bold"
    mpl.rcParams["axes.labelweight"] = "bold"
    
    prediction_start_date = datetime.strptime(prediction_start_date, '%Y-%m-%d')
    prediction_end_date = datetime.strptime(prediction_end_date, '%Y-%m-%d')
    plt.xlim([forecast['ds'].min(),prediction_end_date])
    plt.axvspan(xmin = prediction_start_date , xmax=prediction_end_date, color='grey', alpha=0.2, lw=0)
    plt.plot(forecast['ds'],forecast['y'].cumsum(), color = 'orange', label ='actual values')    
    plt.plot(forecast['ds'],forecast['yhat'].cumsum(), color = '#33adff', label ='predicted values')
    
    plt.xticks(rotation=90, fontsize =18, fontweight='bold')
    plt.yticks(fontsize =18, fontweight='bold') 
    
    plt.xlabel('') 
 
    if normalise:
        plt.ylabel('Cumulative counts per 100k', fontsize =18, fontweight='bold')
        title = '{} in {} Region normalised'.format(outcome_label[outcome],location)
    else:
        plt.ylabel('Cumulative counts', fontsize =18, fontweight='bold')
        title = '{} in {} Region'.format(outcome_label_original[outcome],location)

    plt.title(title,fontsize = 20, fontweight='bold')
    plt.legend(loc='upper left',fontsize=12)
    plt.savefig("plots/"+title+"_its.png", dpi=300,bbox_inches='tight')
    plt.show()
    plt.close()
    

In [ ]:
# Plot Cumulative forecast for each region and outcome
for region in forecast_dict.keys():
    for measure in forecast_dict[region].keys():
        plot_cumulative(location = region,        
                     outcome = measure,
                     forecast = forecast_dict[region][measure],
                     prediction_start_date = prediction_start_date,
                     prediction_end_date = prediction_end_date)

### d. Get Metrics

In [ ]:
def get_metrics(location, outcome, forecast, prediction_start_date, prediction_end_date):
    
    '''
    Function to get the metrics from the forecast.
    
    @params:
    location: Geographical unit (string)
    outcome: Outcome measure (string)
    forecast: Dataframe with the prophet forecast output for the geographical unit and outcome (pd.DataFrame)
    prediction_start_date: Prediction start date (datetime)
    prediction_end_date: Prediction end date (datetime)
    
    '''
    
    prediction_start_date = datetime.strptime(prediction_start_date, '%Y-%m-%d')
    prediction_end_date = datetime.strptime(prediction_end_date, '%Y-%m-%d')

    df_before = forecast[forecast['ds']<prediction_start_date]

    df_after = forecast[(forecast['ds']>=prediction_start_date) & (forecast['ds']<=prediction_end_date)]

    metrics = dict()
    metrics['location'] = location
    metrics['outcome'] = outcome
    
    metrics['mape_before'] = round(np.mean(((df_before['y'] - df_before['yhat'])/df_before['y']).abs()),2)
    metrics['mape_after'] = round(np.mean(((df_after['y'] - df_after['yhat'])/df_after['y']).abs()),2)
    
    metrics['actual_mean_before'] = int(round(df_before['y'].mean(),0))
    metrics['predicted_mean_before'] = int(round(df_before['yhat'].mean(),0))
    metrics['actual_mean_after'] = int(round(df_after['y'].mean(),0))
    metrics['predicted_mean_after'] = int(round(df_after['yhat'].mean(),0))
    
    metrics['actual_median_before'] = int(round(df_before['y'].median(),0))
    metrics['predicted_median_before'] = int(round(df_before['yhat'].median(),0))
    metrics['actual_median_after'] = int(round(df_after['y'].median(),0))
    metrics['predicted_median_after'] = int(round(df_after['yhat'].median(),0))
    
    metrics['mean_change_before'] = np.mean(df_before['change'])
    metrics['wilcoxon_change_before'] = (wilcoxon(df_before['change'] ))
    metrics['mean_change_after'] = np.mean(df_after['change'])
    metrics['wilcoxon_change_after'] = (wilcoxon(df_after['change'] ))
        
    metrics['change_conf_int_before'] = t.interval(alpha=0.95, df=len(df_before['change'])-1, loc=np.mean(df_before['change']), scale=sem(df_before['change']))
    metrics['change_conf_int_after'] = t.interval(alpha=0.95, df=len(df_after['change'])-1, loc=np.mean(df_after['change']), scale=sem(df_after['change']))
    
    metrics['mean_percent_change_before'] = np.mean(df_before['percent_change'])
    metrics['wilcoxon_percent_change_before'] = (wilcoxon(df_before['percent_change']))
    
    metrics['mean_percent_change_after'] = np.mean(df_after['percent_change'])
    metrics['wilcoxon_percent_change_after'] = (wilcoxon(df_after['percent_change']))
    
    metrics['percent_change_conf_int_before'] = t.interval(alpha=0.95, df=len(df_before['percent_change'])-1, loc=np.mean(df_before['percent_change']), scale=sem(df_before['percent_change']))
    metrics['percent_change_conf_int_after'] = t.interval(alpha=0.95, df=len(df_after['percent_change'])-1, loc=np.mean(df_after['percent_change']), scale=sem(df_after['percent_change']))
    
    return metrics

In [ ]:
# Get metrics for each region and outcome
region_metrics ={}
for region in forecast_dict.keys():
    measures_metrics = {}
    for measure in forecast_dict[region].keys():
        itl_metrics = get_metrics(location = region,        
                                     outcome = measure,
                                     forecast = forecast_dict[region][measure],
                                     prediction_start_date = prediction_start_date,
                                     prediction_end_date = prediction_end_date)
        
        measures_metrics[measure] = itl_metrics
    region_metrics[region] = measures_metrics

In [ ]:
region_metrics

In [ ]:
region_metrics['North Central']['Antenatal 4th Visit']

### e. Plot Metrics

In [ ]:
def plot_percent_change(data, prediction_start_date):
    
    '''
    Function to plot the percentage change of the outcomes and the predicted values
    
    @params:
    data: Dataframe with the metrics from the forecast (pd.DataFrame)
    prediction_start_date: Prediction start date (datetime)
    
    '''
    
    df= data[data['month']==prediction_start_date].copy()
    df = df.sort_values(['outcome', 'mean_percent_change_after'], ascending = False)
    
    prediction_start_date = datetime.strptime(prediction_start_date, '%Y-%m-%d')

    sns.set(rc={'figure.figsize':(10,9)})
    mpl.rcParams['font.size'] = 12.0
    mpl.rcParams["font.weight"] = "bold"
    mpl.rcParams["axes.labelweight"] = "bold"

    spot =0
    for outcome in set(df['outcome']): 
        spot +=1
        plt.subplot(3,1,spot)

        x  = df['location'][df['outcome']==outcome]
        y = np.array(list(df['mean_percent_change_after'][df['outcome']==outcome]))
        conf = np.array(list(df['percent_change_conf_int_after'][df['outcome']==outcome]))
        yconf = np.c_[y-conf[:,0],conf[:,1]-y ].T

        plt.plot(y,x,'.', color ='blue')
        plt.axvline(x = 0, linewidth = 1, linestyle ="--", color ='grey')
        title = '{}'.format(outcome_label_original[outcome])
        plt.title(title, loc='center', fontsize=12, fontweight='bold')
        plt.xlabel('percent change in {} counts'.format(y_axis_label[outcome]), fontsize =12, fontweight='bold')
        plt.errorbar(y, x, xerr=yconf, fmt =' ', color ='blue')
    suptitle = 'Mean and 95% CI of Percent Change in Predicted vs Actual Counts ({} {} to December 2020)'.format(prediction_start_date.strftime("%B"),prediction_start_date.year)
    plt.suptitle(suptitle,fontsize=14, fontweight='bold')
    plt.tight_layout(pad=4.0)
    plt.savefig("plots/{}.png".format(suptitle), dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
def plot_diff(data, prediction_start_date):
    '''
    Function to plot the difference of the outcomes and the predicted values
    
    @params:
    data: Dataframe with the metrics from the forecast (pd.DataFrame)
    prediction_start_date: Prediction start date (datetime)
    
    '''
    
    df= data[data['month']==prediction_start_date].copy()
    df = df.sort_values(['outcome', 'mean_change_after'], ascending = False)
    
    prediction_start_date = datetime.strptime(prediction_start_date, '%Y-%m-%d')

    sns.set(rc={'figure.figsize':(11,9)})
    mpl.rcParams['font.size'] = 12.0
    mpl.rcParams["font.weight"] = "bold"
    mpl.rcParams["axes.labelweight"] = "bold"

    spot =0
    for outcome in set(df['outcome']): 
        spot +=1
        plt.subplot(3,1,spot)

        x  = df['location'][df['outcome']==outcome]
        y = np.array(list(df['mean_change_after'][df['outcome']==outcome]))
        conf = np.array(list(df['change_conf_int_after'][df['outcome']==outcome]))
        yconf = np.c_[y-conf[:,0],conf[:,1]-y ].T

        plt.plot(y,x,'.', color ='blue')
        plt.axvline(x = 0, linewidth = 1, linestyle ="--", color ='grey')
        title = '{}'.format(outcome_label_original[outcome])
        plt.title(title, loc='center', fontsize=12, fontweight='bold')
        plt.xlabel('mean of the difference in predicted vs actual {} counts'.format(y_axis_label[outcome]), fontsize =12, fontweight='bold')
        plt.errorbar(y, x, xerr=yconf, fmt =' ', color ='blue')
    suptitle = 'Mean and 95% CI of Difference in Predicted vs Actual Counts ({} {} to December 2020)'.format(prediction_start_date.strftime("%B"),prediction_start_date.year)
    plt.suptitle(suptitle,fontsize=14, fontweight='bold')
    plt.tight_layout(pad=4.0)
    plt.savefig("plots/{}.png".format(suptitle), dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
all_metrics = pd.DataFrame(columns = ['month','location', 'outcome', 
                                      'mape_before', 'mape_after', 
                                      'actual_mean_before', 'predicted_mean_before', 
                                      'actual_mean_after', 'predicted_mean_after', 
                                      'actual_median_before', 'predicted_median_before', 
                                      'actual_median_after', 'predicted_median_after', 
                                      'mean_change_before', 'change_conf_int_before',  'wilcoxon_change_before',
                                      'mean_change_after',  'change_conf_int_after', 'wilcoxon_change_after',
                                      'mean_percent_change_before','percent_change_conf_int_before',  'wilcoxon_percent_change_before',
                                      'mean_percent_change_after','percent_change_conf_int_after', 'wilcoxon_percent_change_after'
       ])

for region in region_metrics.keys():
    for outcome in region_metrics[region].keys():
        metrics = region_metrics[region][outcome]
        metrics['month'] = prediction_start_date
        all_metrics = all_metrics.append(metrics, ignore_index = True)

all_metrics.tail()

In [ ]:
selected_metrics = all_metrics[['month','outcome','location', 
                                'predicted_mean_after','actual_mean_after', 
                                'mean_change_after', 'change_conf_int_after', 'wilcoxon_change_after',
                                'mean_percent_change_after','percent_change_conf_int_after', 'wilcoxon_percent_change_after']]

selected_metrics = selected_metrics.sort_values(['month','outcome', 'location'], ascending = [True,False, True])
selected_metrics['change_conf_int_after'] = [(round(i[0],2),round(i[1],2)) for i in selected_metrics['change_conf_int_after']]
selected_metrics['percent_change_conf_int_after'] = [(round(i[0],2),round(i[1],2)) for i in selected_metrics['percent_change_conf_int_after']]
selected_metrics['wilcoxon_change_after'] = [round(i[1],4) for i in selected_metrics['wilcoxon_change_after']]
selected_metrics['wilcoxon_percent_change_after'] = [round(i[1],4) for i in selected_metrics['wilcoxon_percent_change_after']]
selected_metrics.to_csv('selected_metrics.csv',index=False)
selected_metrics.head()

In [ ]:
plot_diff(selected_metrics, '2020-03-01')

In [ ]:
plot_percent_change(selected_metrics, '2020-03-01')

## 5. Tuning hyperparameters

This step takes at least 2 hours to run, the tuned parameters were saved in a pickle file to be used for later

In [ ]:
cutoff_start='2019-02-01'
cutoff_end='2019-10-01'

In [ ]:
def tune_hyperparameters(location, outcome, data, cutoff_start='2019-02-01', cutoff_end='2019-10-01'):
    '''
    Function to tune the prophet model hyperparameters for the dataset.
    
    @params:
    location: Geographical unit (string)
    outcome: Outcome measure (string)
    forecast: Dataframe with date and the outcome value for the geographical unit and outcome. (pd.DataFrame)
    cutoff_start: Start date (datetime)
    cutoff_end: End date (datetime)
    
    '''

#     df = data[data['displayName'] == outcome].copy() 
    df = data.copy()
    
    cutoff_start = datetime.strptime(cutoff_start, '%Y-%m-%d')
    cutoff_end = datetime.strptime(cutoff_end, '%Y-%m-%d')
    cutoffs = pd.date_range(start=cutoff_start, end=cutoff_end, freq='MS')    
    
    param_grid = {  
        'changepoint_prior_scale': [0.001, 0.01, 0.05, 0.1],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
        'seasonality_mode': ['additive', 'multiplicative'],
    }
    
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here

    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(interval_width=.95, 
                    growth ='linear',
                    yearly_seasonality=False, 
                    weekly_seasonality = False, 
                    daily_seasonality = False,
                    **params
                   ).add_seasonality(
                        name='yearly',
                        period = 365,
                        fourier_order = 5
                   )
        m.fit(df)  
        df_cv = cross_validation(model=m, horizon='90 days', cutoffs=cutoffs, parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses
    tuning_results = tuning_results.sort_values('rmse')
    best_params = all_params[np.argmin(rmses)]
    
    return tuning_results, best_params
    

In [ ]:
# tune the hyperparameters for all regions and outcomes
region_hyperparameters ={}
for region in regions:
    measures_parameters = {}
    for measure in final_columns:
        data = df_copy.loc[df_copy['Region'] == region].copy()[[measure,'Date','Region']]
        data = data[['Date', measure]]
        data.columns = ['ds','y']
        print(region,measure)
        
        tuning_results, best_params = tune_hyperparameters(location = region,
                                                           outcome = measure,
                                                           data = data, 
                                                           cutoff_start = cutoff_start, 
                                                           cutoff_end = cutoff_end)
        
        measures_parameters[measure] = tuning_results, best_params
    region_hyperparameters[region] = measures_parameters

In [ ]:
# Save the tuned hyperparameters to save computation time
with open('tuned_params_nigeria_regions.pkl', 'wb') as handle:
    pickle.dump(region_hyperparameters, handle, protocol=2)

In [ ]:
# Load the previously saved tuned parameters
with open('tuned_params/tuned_params_nigeria_regions.pkl', 'rb') as f:
    region_hyperparameters = pickle.load(f)
region_hyperparameters

In [ ]:
# Preview the tuned parameters
for x in region_hyperparameters.keys():
    for y in region_hyperparameters[x].keys():
        z, best_params = region_hyperparameters[x][y]
        print(x, y, best_params)

# 6) Get forecasts using tuned hyperparameters & Performance Evaluation using Model Cross Validation

#### a. Functions for performance evaluation using Model Cross Validation

In [ ]:
def cross_validate(location, outcome, data, seasonality_mode='additive', changepoint_prior_scale=0.05, seasonality_prior_scale=10.0, cutoff_start='2019-02-01', cutoff_end='2019-10-01'):
    
    '''
    Function for cross validation to evaluate the model's perfomance.
    
    @params:
    location: Geographical unit (string)
    outcome: Outcome measure (string)
    data: Dataframe with date and the outcome value for the geographical unit and outcome. (pd.DataFrame)
    seasonality_mode: tuned prophet model parameter (string)
    changepoint_prior_scale: tuned prophet model parameter (double)
    seasonality_prior_scale: tuned prophet model parameter (double)
    cutoff_start: Prediction start date (datetime)
    cutoff_end: Prediction end date (datetime)
    
    '''
    df = data.copy()  

    m = Prophet(interval_width=.95, 
            growth ='linear',
            yearly_seasonality=False, 
            weekly_seasonality = False, 
            daily_seasonality = False,
            seasonality_mode = seasonality_mode,
            changepoint_prior_scale = changepoint_prior_scale,
            seasonality_prior_scale = seasonality_prior_scale,
            ).add_seasonality(
                name='yearly',
                period = 365,
                fourier_order = 5
            )
    
    m.fit(df)
    
    cutoff_start = datetime.strptime(cutoff_start, '%Y-%m-%d')
    cutoff_end = datetime.strptime(cutoff_end, '%Y-%m-%d')
    cutoffs = pd.date_range(start=cutoff_start, end=cutoff_end, freq='MS')
    
    df_cv = cross_validation(model=m, horizon='90 days', cutoffs=cutoffs)
    
    return df_cv
    

In [ ]:
def plot_cv_metric(location, outcome, df_cv, prediction_start_date):
    
    '''
    Function to plot the cross-validation metrics.
    
    @params:
    location: Geographical unit (string)
    outcome: Outcome measure (string)
    df_cv: Dataframe with date and the outcome value for the geographical unit and outcome. (pd.DataFrame)
    seasonality_mode: tuned prophet model parameter (string)
    changepoint_prior_scale: tuned prophet model parameter (double)
    seasonality_prior_scale: tuned prophet model parameter (double)
    cutoff_start: Prediction start date (datetime)
    
    '''
    sns.set(rc={'figure.figsize':(10,8)})
    sns.set_style("white")
    mpl.rcParams['font.size'] = 8.0
    mpl.rcParams["font.weight"] = "bold"
    mpl.rcParams["axes.labelweight"] = "bold"
    
    plot_cross_validation_metric(df_cv, metric='mape')
    plt.xticks(fontsize =18, fontweight='bold')
    plt.yticks(fontsize =18, fontweight='bold') 
    plt.xlabel('Horizon',fontsize =18, fontweight='bold')
    plt.ylabel('MAPE', fontsize =18, fontweight='bold')
        
    prediction_start_date = datetime.strptime(prediction_start_date, '%Y-%m-%d')
    title = 'MAPE for {} in {} ({} {})'.format(outcome_label_original[outcome], location, prediction_start_date.strftime("%B"),prediction_start_date.year)
    
    plt.title(title,fontsize = 20, fontweight='bold')
    plt.savefig("plots/{}.png".format(title),  dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()    

#### b. Run the ITS & Cross-validation step

In [ ]:
forecast_dict_tuned = {}

for region in regions:
    measures_forecast_tuned = {}
    forecast_dict_tuned[region] = {}
    for measure in final_columns:
        forecast_dict_tuned[region][measure] = {}
        data = df_copy.loc[df_copy['Region'] == region].copy()[[measure,'Date','Region']]
        data = data[['Date', measure]]
        data.columns = ['ds','y']
        
        x, best_params = region_hyperparameters[region][measure]

        # Get forecast
        forecast = get_forecasts(location = region,
                                          outcome = measure,
                                          data = data, 
                                          seasonality_mode = best_params['seasonality_mode'],
                                          changepoint_prior_scale = best_params['changepoint_prior_scale'],
                                          seasonality_prior_scale = best_params['seasonality_prior_scale'],
                                          prediction_start_date = prediction_start_date,
                                          prediction_end_date = prediction_end_date)

        forecast_dict_tuned[region][measure]['forecast'] = forecast


        # Get Metrics
        itl_metrics_tuned = get_metrics(location = region,        
                     outcome = measure,
                     forecast = forecast,
                     prediction_start_date = prediction_start_date,
                     prediction_end_date = prediction_end_date)

        forecast_dict_tuned[region][measure]['metrics'] = itl_metrics_tuned

        #cross validation
        df_cv = cross_validate(location = region,
                               outcome = measure,
                               data = data, 
                               seasonality_mode = best_params['seasonality_mode'],
                               changepoint_prior_scale = best_params['changepoint_prior_scale'],
                               seasonality_prior_scale = best_params['seasonality_prior_scale'],
                               cutoff_start = cutoff_start, 
                               cutoff_end = cutoff_end)

        forecast_dict_tuned[region][measure]['df_cv'] = df_cv


        df_p = performance_metrics(df_cv)

        forecast_dict_tuned[region][measure]['df_p'] = df_p


        # plots
        plot_its(location = region,        
                             outcome = measure,
                             forecast = forecast,
                             prediction_start_date = prediction_start_date,
                             prediction_end_date = prediction_end_date)

        plot_cumulative(location = region,        
                             outcome = measure,
                             forecast = forecast,
                             prediction_start_date = prediction_start_date,
                             prediction_end_date = prediction_end_date)

        plot_cv_metric(location = region,
                                   outcome = measure,
                                   df_cv = df_cv,
                                   prediction_start_date = prediction_start_date)


In [ ]:
for x in region_hyperparameters.keys():
    for y in region_hyperparameters[x].keys():
        z, best_params = region_hyperparameters[x][y]
        print(x, y, best_params)

In [ ]:
# Save the output to save computation time
with open('forecast_dict_tuned_nigeria_regions.pkl', 'wb') as handle:
    pickle.dump(forecast_dict_tuned, handle, protocol=2)

In [ ]:
def create_df_metrics(region_metrics:dict(), prediction_start_date = prediction_start_date):
    all_metrics = pd.DataFrame(columns = ['month','location', 'outcome', 
                                      'mape_before', 'mape_after', 
                                      'actual_mean_before', 'predicted_mean_before', 
                                      'actual_mean_after', 'predicted_mean_after', 
                                      'actual_median_before', 'predicted_median_before', 
                                      'actual_median_after', 'predicted_median_after', 
                                      'mean_change_before', 'change_conf_int_before',  'wilcoxon_change_before',
                                      'mean_change_after',  'change_conf_int_after', 'wilcoxon_change_after',
                                      'mean_percent_change_before','percent_change_conf_int_before',  'wilcoxon_percent_change_before',
                                      'mean_percent_change_after','percent_change_conf_int_after', 'wilcoxon_percent_change_after'
       ])

    for region in region_metrics.keys():
        for outcome in region_metrics[region].keys():
            metrics = region_metrics[region][outcome]['metrics']
            metrics['month'] = prediction_start_date
            all_metrics = all_metrics.append(metrics, ignore_index = True)
            


    selected_metrics = all_metrics[['month','outcome','location', 
                                'predicted_mean_after','actual_mean_after', 
                                'mean_change_after', 'change_conf_int_after', 'wilcoxon_change_after',
                                'mean_percent_change_after','percent_change_conf_int_after', 'wilcoxon_percent_change_after']]

    selected_metrics = selected_metrics.sort_values(['month','outcome', 'location'], ascending = [True,False, True])
    selected_metrics['change_conf_int_after'] = [(round(i[0],2),round(i[1],2)) for i in selected_metrics['change_conf_int_after']]
    selected_metrics['percent_change_conf_int_after'] = [(round(i[0],2),round(i[1],2)) for i in selected_metrics['percent_change_conf_int_after']]
    selected_metrics['wilcoxon_change_after'] = [round(i[1],4) for i in selected_metrics['wilcoxon_change_after']]
    selected_metrics['wilcoxon_percent_change_after'] = [round(i[1],4) for i in selected_metrics['wilcoxon_percent_change_after']]
    selected_metrics.to_csv('selected_metrics.csv',index=False)
    
    return selected_metrics

In [ ]:
# Create important metrics
selected_important_metrics = create_df_metrics(forecast_dict_tuned,prediction_start_date)

# plot the important metrics
plot_diff(selected_important_metrics, prediction_start_date)
plot_percent_change(selected_important_metrics, prediction_start_date)